In [ ]:
import pandas as pd #the csv reading library
import matplotlib.pyplot as plt #the ploting library
import glob #to get file names
import scipy.optimize
from scipy.optimize import curve_fit
from scipy import interpolate
from scipy.interpolate import pchip_interpolate
import numpy as np

In [ ]:
# This returns a list of names (strings) of files that have the .txt extension. 
# You should provide the path
files = glob.glob('../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB/*.txt')

In [ ]:
#first Identify what wavegide was used then uncomment the correct waveguide 
X = .488 # 50 cm lwcc number 3 SN: 

In [ ]:
#SAMPLE OPENS FILES 
def open_file(name):
    # returns a datafame from a raw file
    # input: filename (string)
    # output: pandas dataframe 
    df = pd.read_csv(name, skiprows=14, sep='\t', names=['wl', 'abs'], )
    return df

In [ ]:
#SAMPLE ABS TRANSFORM AND BASLINE CORRECTION 
def abs_base_cor(name):
    # unit conversion
    df['abs'] = df['abs']*(2.303/X)
    # baseline correction
    bcorr = df[(df['wl']>=700) & (df['wl']<=750)]['abs'].mean() # a float
    df['abs'] = df['abs'] - bcorr
    return df

In [ ]:
#SAMPLE PCHIPS DATA INTO WHOLE WAVELENTHS 
def Pchip(name):
    #PCHIP 1-D monotonic cubic interpolation.
    #x and y are arrays of values used to approximate some function f, with y = f(x).
    #The interpolant uses monotonic cubic splines to find the value of new points. 
    #PCHIP stands for Piecewise Cubic Hermite Interpolating Polynomial
    x = df['wl']
    y = df['abs']
    #dose the interplation 
    pchip_obj1 = scipy.interpolate.PchipInterpolator(x, y, axis=0, extrapolate=None)
    #pulls the wavelenths data point from the interplation 
    xi = np.arange(189,881,1)
    #pulls the abs values from the wavelenths from the interplation
    yi = pchip_obj1(xi)
    #this turns it back into a dataframe 
    d = {'wl': xi, 'abs': yi}
    df = pd.DataFrame(d)
    return df

In [ ]:
#FINAL FUNTION 
#see above for the breakdown of diff parts 
def GET_CDOM_DATA(name):
    df = pd.read_csv(name, skiprows=14, sep='\t', names=['wl', 'abs'], )
    df['abs'] = df['abs']*(2.303/X)
    bcorr = df[(df['wl']>=700) & (df['wl']<=750)]['abs'].mean() # a float
    df['abs'] = df['abs'] - bcorr
    x = df['wl']
    y = df['abs'] 
    pchip_obj1 = scipy.interpolate.PchipInterpolator(x, y, axis=0, extrapolate=None)
    xi = np.arange(189,881,1)
    yi = pchip_obj1(xi)
    d = {'wl': xi, 'abs': yi}
    df = pd.DataFrame(d)
    return df

In [ ]:
#TEST FUNCTION 
GET_CDOM_DATA('../../../CDOM_DATA/BLU22/BC_ABS_BLU22_2226_GB/222601_Absorbance__0__00.txt')

In [ ]:
for f in files:
    df = GET_CDOM_DATA(f)
    ID = f.split("\\")[-1][0:-4]
    


Ok... so here is the issue 
there are many scanns of the same sample 
after I do the data processing I need to average them by sample ID
sample ID that is the six diget code in the file name 
then I need to export it to jordon 

so I need a col with the sample ID
and I need a col with each waveleht and its abs value 

then I can move on to other forms of data proccesing... 

In [ ]:
df = pd.read_csv(files[1], skiprows=14, names=['wl', 'abs'], sep='\t')
df['abs'] = df['abs']*(2.303/.488) #the denominator is spacific the the waveguide
bcorr = df[(df['wl']>=700) & (df['wl']<=750)]['abs'].mean()
df['abs'] = df['abs']-bcorr

ID = files[1].split("\\")[-1][-0][1:-4]
df.rename({'abs': ID})

ID

In [ ]:
ID = f.split("\\")[-1][0:-4]
ID

In [ ]:
#THIS IS A GRAPH OF ALL THE FILES ABSORPANCE
#creates an empty list
frames = []

#loops through the list of names
for f in files:
    #opens file with above function
    df = GET_CDOM_DATA(f)
    # updates the list created above with the new dataframe
    frames.append(df)
#frames is a list of dataframes

#concatenates all dataframes in one single dataframe
full = pd.concat(frames, axis=1, ignore_index=False)

full

In [ ]:
#SAMPLE RENAMES DATA FRAME FROM FILE NAME
#DOSE NOT WORK 
def rename_data_frame(name):
    ID = f.split("\\")[-1][0:-4]
    df.rename({ID})
     return df

In [ ]:
#we too three scans of the samples, and we want to average the scanns 
#we will use the underscore in the file name to separete the averages 
# set the nummber to the approprate number of undercores in the same to seperate by the sample ID or Standterd 

sample = np.unique([f.split('_')[5] for f in files])
sample